<a href="https://colab.research.google.com/github/aayush2710/CTW2019-IDP/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!ls "/content/gdrive/My Drive/CTW_2019"

In [ ]:
import numpy as np # linear algebra
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, ReLU, Input, Dropout, Conv2D, AveragePooling2D, Flatten, AveragePooling1D
from keras.optimizers import RMSprop
import numpy as np
import os
import h5py
import pandas as pd
import time
import matplotlib.pyplot as plt
import tensorflow as tf

Using TensorFlow backend.


In [ ]:
'''#getting and processing data
#h_Estimated   ---> shape = (2, 924, 16, 17486)
#r_postition ---> shape = (3, 17486)
#SNR_Est    ---> shape = (16, 17486)
'''

In [ ]:
#Reading the dataset

f = h5py.File('/content/gdrive/My Drive/CTW_2019/r_Position_CTW_Train.h5','r')
pos = f['r_Position']
f = h5py.File('/content/gdrive/My Drive/CTW_2019/SNR_CTW_Train.h5','r')
SNR = f['SNR_Est']

In [ ]:
print(H.shape)
H = np.array(H)
pos = np.array(pos)
#data = H.reshape((17486,16,924,2))
#Y = pos.reshape((17486,3))

In [ ]:
data = H.swapaxes(0,-1)
print(data.shape)

print(data[1].shape)
print(H[1,:,:,1])

In [ ]:
pos = pos.swapaxes(0,1)
print(pos.shape)
print(pos[1])

In [ ]:
#dividing the data for testing and training

from sklearn.model_selection import train_test_split
X_train,X_test = train_test_split(data, test_size=0.1, random_state=42, shuffle=True)
y_train,y_test = train_test_split(pos, test_size=0.1, random_state=42, shuffle=True)
#print(H_new.shape,pos.shape)

print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)



In [ ]:
#CNN model

def cnn2d():
  inp = Input(shape=(924,16,2))
  x = Conv2D(32, (3,3), padding="same",activation='relu')(inp)
  x = AveragePooling2D(pool_size=(4,1))(x)
  x =Dropout(0.25)(x)
  x = Conv2D(32, (3,3), padding="same",activation='relu')(x)
  x = AveragePooling2D(pool_size=(4,1))(x)
  x = Dropout(0.25)(x)

  x = Flatten()(x)
  x = Dense(256,activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(256,activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(256,activation='relu')(x)
  x  = Dropout(0.2)(x)

  out = Dense(3)(x)

  
  return Model(inp,out)


In [ ]:
m1 = cnn2d()
print(m1.summary())

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
# Training 

from keras.callbacks import EarlyStopping, ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', patience = 20)
checkpointer = ModelCheckpoint('/content/gdrive/My Drive/CTW_2019/best_cnn.h5', verbose=1, save_best_only=True)

opt = RMSprop(learning_rate=0.001, decay=1e-6)
m1.compile(loss='mean_squared_error',
              optimizer=opt,
              )

m1.fit(X_train, y_train,
              batch_size=20,
              epochs=1000,
              validation_split=0.12,
              shuffle=True, callbacks=[early_stopping, checkpointer])
    

# Score trained model.
scores = m1.evaluate(X_test, y_test, verbose=1)


In [ ]:
def rmse(p1, p2):
  return np.sqrt(np.sum((p1-p2)**2, axis = 1))

In [ ]:
#Evaluating

m2 = load_model('/content/gdrive/My Drive/CTW_2019/best_cnn.h5')
tt = m2.predict(X_test)
print(tt)
print(y_test)
gg  = (rmse(tt,y_test))

plt.hist(gg, bins=20)
plt.show()
plt.hist(np.log(gg), bins=20)

In [ ]:


ttt = m2.predict(X_test)

dd = rmse(ttt, y_test)

print(np.mean(dd))